# Import Packages


In [89]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

import numpy as np
from scipy.stats import qmc
from itertools import product

# Config

In [90]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 6302,      # Your seed number, you can pick your lucky number. :)
    "gen_num": 100, 
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 6000,     # Number of epochs.            
    'batch_size': 16, 
    'learning_rate': 1e-4,              
    'early_stop': 800,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

# Function Definition 
x: 原圖  
p: 所下的參數  
y: 處理過的圖 

### $y = f(x, p)$
假設x只有兩個pixel  
$y_0 = p_0 * x_0^{2} + p_1 * x_1 + p_2$  
$y_1 = p_0 * x_1^{2} + p_1 * x_0 + x_1$  

In [91]:
def f(x, p): # x,p -> y
    x = np.array(x).T
    p = np.array(p).T
    y0 = p[0]*x[0]**2 + p[1]*x[1] + p[2]
    y1 = p[0]*x[1]**2 + p[1]*x[0] + x[1]
    return np.array([y0, y1]).T

# ex
# x = [[1, 2]]
# p = [[0.1, 0.2, 0.3], [0.2, 0.2, 0.3], [0.3, 0.2, 0.3]] # parameters are normalized to [0, 1]

# Dataset

In [92]:
class MyDataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y, p):
        self.x = torch.FloatTensor(x)
        self.y = torch.FloatTensor(y)
        self.p = torch.FloatTensor(p)

    def __getitem__(self, idx):
            return self.x[idx], self.y[idx], self.p[idx]

    def __len__(self):
        return len(self.x)
    
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return train_set, valid_set

# Generate Dataset
* 初步實驗 假設都是同一張圖 (x固定)

In [93]:
x = [[1, 2]]*config['gen_num'] # 假設圖固定，只修改參數
sampler = qmc.LatinHypercube(d=3) # 假設p有3個參數
p = sampler.random(n=config['gen_num']) # 隨機產生 gen_num 組

dataset = MyDataset(x, f(x, p), p)
train_dataset, valid_dataset = train_valid_split(dataset, config['valid_ratio'], config['seed'])

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)

# Model

In [94]:
class Proxy_Model(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Proxy_Model, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, output_dim)
        )

    def forward(self, x):
        x = self.layers(x)
        return x

# Step1: Train Proxy Model

In [95]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.
    optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate']) 

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, early_stop_count = config['n_epochs'], math.inf, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        for x, y, p in train_loader:
            optimizer.zero_grad()               # Set gradient to zero.
            # 假設都是同一張圖(只輸入參數 p )
            x, y = p.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)             
            loss = criterion(pred, y)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            loss_record.append(loss.detach().item())

        mean_train_loss = np.round(sum(loss_record)/len(loss_record), 6)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y, p in valid_loader:
            x, y = p.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = np.round(sum(loss_record)/len(loss_record), 6)
        if epoch%100==0:
            print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.6f}, Valid loss: {mean_valid_loss:.6f}')

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            # print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return
        
proxy_model = Proxy_Model(input_dim=3, output_dim=2).to(device) # put your model and data on the same computation device.
trainer(train_loader, valid_loader, proxy_model, config, device)

Epoch [1/6000]: Train loss: 10.940766, Valid loss: 13.738888
Epoch [101/6000]: Train loss: 8.572897, Valid loss: 11.356069
Epoch [201/6000]: Train loss: 4.753461, Valid loss: 5.920917
Epoch [301/6000]: Train loss: 1.232263, Valid loss: 1.862384
Epoch [401/6000]: Train loss: 0.555594, Valid loss: 0.814832
Epoch [501/6000]: Train loss: 0.475844, Valid loss: 0.529487
Epoch [601/6000]: Train loss: 0.417189, Valid loss: 0.427874
Epoch [701/6000]: Train loss: 0.357148, Valid loss: 0.341171
Epoch [801/6000]: Train loss: 0.296983, Valid loss: 0.362515
Epoch [901/6000]: Train loss: 0.237790, Valid loss: 0.232322
Epoch [1001/6000]: Train loss: 0.181074, Valid loss: 0.165264
Epoch [1101/6000]: Train loss: 0.128751, Valid loss: 0.162789
Epoch [1201/6000]: Train loss: 0.083609, Valid loss: 0.101525
Epoch [1301/6000]: Train loss: 0.049098, Valid loss: 0.050481
Epoch [1401/6000]: Train loss: 0.026236, Valid loss: 0.019444
Epoch [1501/6000]: Train loss: 0.012539, Valid loss: 0.015185
Epoch [1601/6000]

# Step2: Fix Proxy Model and Tune

In [99]:
class Tune_Proxy_Model(nn.Module):
    def __init__(self, input_dim, proxy_model):
        super(Tune_Proxy_Model, self).__init__()
        self.proxy_model = proxy_model
        self.tuning_fn = nn.Linear(input_dim, input_dim, bias=False)

    def forward(self, p):
        p = self.tuning_fn(p)
        y = self.proxy_model(p)
        return y
    
# Load Proxy Model
proxy_model = Proxy_Model(input_dim=3, output_dim=2).to(device)
proxy_model.load_state_dict(torch.load(config['save_path']))

tune_model = Tune_Proxy_Model(3, proxy_model)

In [100]:
def tuning(model, config, device, x, y):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.
    optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate']) 

    n_epochs, best_loss, early_stop_count = 10000, math.inf, 0

    for epoch in range(n_epochs):
        model.tuning_fn.train()
        model.proxy_model.eval() 
        # fix proxy model
        for param in model.proxy_model.parameters():
            param.requires_grad = False
        
        loss_record = []

        optimizer.zero_grad()               # Set gradient to zero.
        init_p = [1, 1, 1] # init p
        init_p, y = torch.FloatTensor(init_p).to(device), torch.FloatTensor(y).to(device)   # Move your data to device. 
        pred = model(init_p)             
        loss = criterion(pred, y)
        loss.backward()                     # Compute gradient(backpropagation).
        optimizer.step()                    # Update parameters.
        model.tuning_fn.weight.data.clamp_(min=0, max=1)
        loss_record.append(loss.detach().item())

        mean_train_loss = np.round(sum(loss_record)/len(loss_record), 6)
        
        if epoch%500==0:
            print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}')

        if mean_train_loss < best_loss:
            best_loss = mean_train_loss
            early_stop_count = 0
        else: 
            early_stop_count += 1

        

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            break
        
    return model.tuning_fn(init_p).detach().numpy()

In [101]:
x = [1, 2]
ans_p = [0.1, 0.2, 0.3] # 自己假設的解答
y = f(x, ans_p)
pred_p = tuning(tune_model, config, device, x, y)

Epoch [1/10000]: Train loss: 1.1900
Epoch [501/10000]: Train loss: 0.8884
Epoch [1001/10000]: Train loss: 0.5986
Epoch [1501/10000]: Train loss: 0.3858
Epoch [2001/10000]: Train loss: 0.2360
Epoch [2501/10000]: Train loss: 0.1349
Epoch [3001/10000]: Train loss: 0.0760
Epoch [3501/10000]: Train loss: 0.0449
Epoch [4001/10000]: Train loss: 0.0269
Epoch [4501/10000]: Train loss: 0.0146
Epoch [5001/10000]: Train loss: 0.0068
Epoch [5501/10000]: Train loss: 0.0026
Epoch [6001/10000]: Train loss: 0.0007
Epoch [6501/10000]: Train loss: 0.0002
Epoch [7001/10000]: Train loss: 0.0000
Epoch [7501/10000]: Train loss: 0.0000
Epoch [8001/10000]: Train loss: 0.0000

Model is not improving, so we halt the training session.


In [102]:
print(f(x, ans_p))
print(proxy_model(torch.FloatTensor(ans_p).to(device)))
print(tune_model.proxy_model(torch.FloatTensor(ans_p).to(device)))

[0.8 2.6]
tensor([0.7980, 2.5983])
tensor([0.7980, 2.5983])


In [103]:
print("實際的參數: ", ans_p)
print("預測的參數: ", pred_p)
print(f(x, ans_p))
print(f(x, pred_p))

實際的參數:  [0.1, 0.2, 0.3]
預測的參數:  [5.7339117e-02 3.7240133e-01 4.2605243e-05]
[0.8 2.6]
[0.80218438 2.60175779]
